In [5]:
import surprise

In [6]:
import tensorflow as tf

In [497]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate, train_test_split
import zipfile
from surprise import Reader, Dataset, SVD
from surprise import accuracy
import random
from random import randint
import re
import pandas as pd
import copy
import operator

<h4> Read in data

In [535]:
# Read data into an array of strings
def read_data(extended = False):
    
    if(extended):
        with open('./new_data.data') as f:
            all_movies = f.readlines()

        # Prepare the data to be used in Surprise
        reader = Reader(line_format='user item rating timestamp', sep='\t')
        data = Dataset.load_from_file('./new_data.data', reader=reader)
        
    else:
        with open('./ml-100k/u.data') as f:
            all_movies = f.readlines()

        # Prepare the data to be used in Surprise
        reader = Reader(line_format='user item rating timestamp', sep='\t')
        data = Dataset.load_from_file('./ml-100k/u.data', reader=reader)
    
    return all_movies, data

In [536]:
# all_lines, data = read_data()

In [10]:
def create_dataframe(data):
    data = [ x.replace('\t', ', ').replace('\n', '') for x in data ]

    df = pd.DataFrame([sub.split(",") for sub in data])
    df.rename(columns={0:'userID', 1:'movieID', 2:'rating', 3: 'timestamp'}, 
                         inplace=True)
    df = df.drop(columns=['timestamp'])
    return df

In [11]:
def strip_content(data):
    r_unwanted = re.compile("[\n\t\r]")
    return r_unwanted.sub(",", data)

In [133]:
# df_final = create_dataframe(all_lines)

In [134]:
# df_final = df_final.astype('int')

Outdated

In [18]:
# def create_dataframe(data):
#     df_data = pd.DataFrame(data)

#     df_data[0] = df_data[0].apply(strip_content)

#     foo = lambda x: pd.Series([i for i in (x.split(','))])

#     df_final = df_data[0].apply(foo)

#     df_final.rename(columns={0:'userID', 1:'movieID', 2:'rating', 3: 'timestamp'}, 
#                      inplace=True)

#     df_final = df_final.drop(columns=[4])
    
#     return df_final

Grid Search KNN

In [19]:
# sim_options = {
#     "name": ["msd", "cosine"],
#     "min_support": [3, 4, 5],
#     "user_based": [False, True],
# }

# param_grid = {"sim_options": sim_options}

# gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
# gs.fit(data)

# print(gs.best_score["rmse"])
# print(gs.best_params["rmse"])

<h4> Grid search for best params </h4>

In [20]:
# param_grid = {
#     "n_epochs": [5, 10],
#     "lr_all": [0.002, 0.005],
#     "reg_all": [0.1, 0.2, 0.4, 0.6]
# }
# gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

# gs.fit(data)

# print(gs.best_score["rmse"])
# print(gs.best_params["rmse"])

<h4> SVD algo chosen, with best params obtained from GS

In [21]:
algo = SVD(n_epochs=10, lr_all=0.005, reg_all=0.4)

<h4>Cross validation, 5 folds

In [22]:
# cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

<h4>Create train and test set, apply predictions on test set

In [23]:
def RMSE_predict_train_test(data):
    # sample random trainset and testset
    # test set is made of 25% of the ratings.
    trainset, testset = train_test_split(data, test_size=.25)

    # We'll use the famous SVD algorithm.
    algo = SVD(n_epochs=10, lr_all=0.005, reg_all=0.4)

    # Train the algorithm on the trainset, and predict ratings for the testset
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Then compute RMSE
    return accuracy.rmse(predictions)

<h4> Training Time
   

In [24]:
import timeit

def training_time(data):
    start = timeit.default_timer()

    trainset = data.build_full_trainset()

    algo.fit(trainset)


    stop = timeit.default_timer()

    print('Time: ', stop - start)  

In [25]:
# training_time(data_other)

<h4>Predict scores for all users for all movies. Test the speed of the predictions.

In [26]:
def predict_scores(data):
    # Build Training set. Needed to fit to create model.
    trainset = data.build_full_trainset()
    
    algo.fit(trainset)
    
    # Get all the user and item IDs
    user_ids = trainset.all_users()
    item_ids = trainset.all_items()

    # Create empty list to store predictions
    ratings = []
    
    # For loop, estimate rating of each user for every movie.
    for user_id in user_ids:
        for item_id in item_ids:
            
            prediction = algo.predict(str(user_id), str(item_id)).est
#             print(prediction)
            ratings.append(prediction)
            
    return ratings

<h4> Time to predict all ratings

In [27]:
# import timeit

# start = timeit.default_timer()


# ratings = predict_scores(data_other)
    
# stop = timeit.default_timer()

# print('Time: ', stop - start)  

<h4> Create Dataframe

In [136]:
def predict_scores_dict(data):
    # Build Training set. Needed to fit to create model.
    print("build trainingset")
    trainset = data.build_full_trainset()
    
    print("training algo")
    algo.fit(trainset)
    
    # Get all the user and item IDs
    user_ids = trainset.all_users()
    item_ids = trainset.all_items()
    
    # Create empty list to store predictions
    ratings = {}
    ratings_list = []
    
    print("start prediction")
    # For loop, estimate rating of each user for every movie.
    for user_id in user_ids:
        for item_id in item_ids:
            
            prediction = algo.predict(str(user_id), str(item_id)).est
#             print(prediction)
            ratings['userID'] = int(user_id)
            ratings['movieID'] = int(item_id)
            ratings['rating'] = prediction
            
            ratings_list.append(ratings)
            
            ratings = {}
            
    return ratings_list

In [137]:
# import timeit

# start = timeit.default_timer()

# predicted_scores_dict =  predict_scores_dict(data)
    
# stop = timeit.default_timer()

# print('Time: ', stop - start)  


In [194]:
# complete_prediction = pd.DataFrame(predicted_scores_dict)

In [ ]:
# complete_prediction = []
# complete_prediction = pd.DataFrame(complete_pr²ediction)
# for i in range(0, len(predicted_scores_dict), 10000000):
#     print("appending",i, i+10000000)
#     complete_prediction = complete_prediction.append(predicted_scores_dict[i:i+10000000])

<h4> Create extended dataset

In [ ]:
def create_data(data):
    trainset = data.build_full_trainset()

    user_ids = trainset.all_users()
    item_ids = trainset.all_items()

    data_list = []
    # Create new movies (168200 in total)
    for movie in range(item_ids[-1]+1, item_ids[-1]*100):
        # For every movie, there will be 100 users rating the movie
        user_generated = [randint(0, user_ids[-1]) for p in range(0, 100)]
        for user in user_generated:
            # Create a random generated score for the movies
            new_data = str(user)+'\t'+str(movie)+'\t'+str(random.randint(1,5))+'\t'+'NaN\n'

            data_list.append(new_data)
    return data_list

In [ ]:
# data_newest = create_data(data)

Create the new data (as a list)

In [ ]:
# new_data = []

# new_data.extend(all_lines)
# new_data.extend(data_newest)

Check difference in sizes


In [ ]:
# len(data_newest)/len(all_lines)

<h4> Search Function </h4>

In [109]:
def drop_dups(userID, df_final, predicted_scores_df):

    df_user0_predicted = predicted_scores_df[predicted_scores_df['userID'] == userID]
    # df_user0_predicted.sort_values('rating', ascending=False)
    df_user0_predicted.reset_index(drop=True, inplace=True)

    df_user0_original = df_final[df_final['userID'] == userID]
    df_user0_original.reset_index(drop=True, inplace=True)

    dfs_dictionary = {'DF1':df_user0_predicted,'DF2':df_user0_original}
    df3=pd.concat(dfs_dictionary)
    df3=df3.drop_duplicates(subset=['userID', 'movieID'],keep=False)
    
    return df3

In [604]:
ascending_ratings = drop_dups(84, df_final2, df_predicted_scores2).sort_values('rating', ascending=False)

In [605]:
ascending_ratings

userID  movieID    rating
DF1 169      84      169  4.423836
    483      84      483  4.361799
    114      84      114  4.345114
    603      84      603  4.329419
    50       84       50  4.320596
...         ...      ...       ...
    743      84      743  2.741580
    368      84      368  2.739369
    424      84      424  2.688997
    890      84      890  2.655195
    688      84      688  2.653239

[1614 rows x 3 columns]

In [169]:
def best_movies_for_user(userID, amount_of_movies, df_final, predicted_scores_df):
    ascending_ratings = drop_dups(userID, df_final, predicted_scores_df).sort_values('rating', ascending=False)
    
    return ascending_ratings[0:amount_of_movies]

In [386]:
def create_dict_movie_genres():
        # def determine_genre():
    with open('./ml-100k/u.genre') as f:
        all_genres = f.readlines()
    movie_genres = {}

    for i in range(len(all_genres)-1):
        split_genres = all_genres[i].split('|')
        movie_genres[int(split_genres[1].split("\n")[0])] = split_genres[0]
        
    return movie_genres

In [378]:
def genre_per_movie():
    with open('./ml-100k/u.item') as f:
        movie_details = f.readlines()

    movie_details_dict = {}

    for i in range(len(movie_details)):
        genre_list = ([pos for pos, char in enumerate(movie_details[i][-39:-2]) if char == '1'])
        genre_list = np.array(genre_list)
        genre_list = genre_list//2
        genre_list = genre_list.tolist()
        
        movie_details_dict[movie_details[i].split('|')[1]] = genre_list

    return movie_details_dict

In [393]:
def number_to_movie():
    with open('./ml-100k/u.item') as f:
        movie_details = f.readlines()
        
    movie_details_dict = {}
    
    for i in range(len(movie_details)):
        movie_details_dict[int(movie_details[i].split('|')[0])] = movie_details[i].split('|')[1]
    
    return movie_details_dict

In [139]:
def insert_into_dataframe(predicted_scores_dict):
    complete_prediction = []
    complete_prediction = pd.DataFrame(complete_prediction)
    for i in range(0, len(predicted_scores_dict), 10000000):
        print("appending",i, i+10000000)
        complete_prediction = complete_prediction.append(predicted_scores_dict[i:i+10000000])
        
    return complete_prediction

In [141]:
def prepare_data():
    all_lines, data = read_data()
    
    print("create df_final")
    df_final = create_dataframe(all_lines).astype('int')
    
    print("predict scores")
    predicted_scores_dict =  predict_scores_dict(data)
    
    df_predicted_scores = insert_into_dataframe(predicted_scores_dict)
#     df_predicted_scores = pd.DataFrame(predicted_scores_dict)
    
    return df_final, df_predicted_scores

In [451]:
def convert_to_titles(best_movies):
    i=0
    for movieID in best_movies['movieID']:
        best_movies['movieID'][i] = number_to_movie[int(movieID)]
        i+=1
        
    return best_movies

In [182]:
# df_final, df_predicted_scores = prepare_data()

In [202]:
df_final2, df_predicted_scores2 = prepare_data()

create df_final
predict scores
build trainingset
training algo
start prediction
appending 0 10000000


In [656]:
best_movies = best_movies_for_user(102,10,df_final2, df_predicted_scores2)
best_movies = best_movies.astype('str')
best_movies = convert_to_titles(best_movies)

In [657]:
best_movies

userID                                            movieID  \
DF1 408    102                              Close Shave, A (1995)   
    169    102                         Wrong Trousers, The (1993)   
    318    102                            Schindler's List (1993)   
    64     102                   Shawshank Redemption, The (1994)   
    483    102                                  Casablanca (1942)   
    114    102  Wallace & Gromit: The Best of Aardman Animatio...   
    12     102                         Usual Suspects, The (1995)   
    603    102                                 Rear Window (1954)   
    178    102                                12 Angry Men (1957)   
    513    102                              Third Man, The (1949)   

                     rating  
DF1 408  3.6335415446486667  
    169  3.6037126187156665  
    318   3.595659447219115  
    64    3.570147970269001  
    483   3.538779696909846  
    114  3.5355298599811293  
    12    3.530441539690147  
    603  3.5042400653287857  
    178  3.4950832889092673  
    513  3.4763355605864126

In [658]:
def best_genre_for_user(userID, amount_of_movies):
    best_movies = best_movies_for_user(userID,amount_of_movies,df_final2, df_predicted_scores2)    

    list_of_genres = []
    for movieID in best_movies['movieID']:
        list_of_genres.extend(genre_per_movie[number_to_movie[int(movieID)]])

    sorted_genres = {value: len(list(freq)) for value, freq in groupby(sorted(list_of_genres))}
    
    best_genre = max(sorted_genres, key=sorted_genres.get)

    return movie_genres[best_genre]

In [660]:
best_genre_for_user(102, 10)

'Drama'

In [510]:
# movie_genres = create_dict_movie_genres()
# genre_per_movie = genre_per_movie()
# number_to_movie = number_to_movie()